# YouTube Video Summarizer using Groq

This notebook demonstrates how to create a YouTube video summarizer that:
1. Takes a YouTube URL as input
2. Extracts the video's transcription
3. Uses Groq to generate a concise summary

## Setup and Requirements
First, we'll install the required packages:

In [ ]:
!pip install youtube-transcript-api groq python-dotenv

## Import Required Libraries

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import groq
import os
from dotenv import load_dotenv
from urllib.parse import urlparse, parse_qs

# Load environment variables
load_dotenv()

# Initialize Groq client
client = groq.Groq(
    api_key=os.getenv('GROQ_API_KEY')
)

## Helper Functions

In [ ]:
def extract_video_id(url):
    """Extract YouTube video ID from URL"""
    parsed_url = urlparse(url)
    if parsed_url.hostname == 'youtu.be':
        return parsed_url.path[1:]
    if parsed_url.hostname in ('www.youtube.com', 'youtube.com'):
        if parsed_url.path == '/watch':
            return parse_qs(parsed_url.query)['v'][0]
    return None

def get_transcript(video_id):
    """Get transcript for a YouTube video"""
    try:
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        return ' '.join([t['text'] for t in transcript_list])
    except Exception as e:
        print(f"Error getting transcript: {e}")
        return None

def summarize_text(text):
    """Summarize text using Groq"""
    prompt = f"""Please provide a comprehensive summary of the following video transcript. 
    Focus on the main points, key insights, and important conclusions:

    {text}

    Please structure the summary with:
    1. Main Topic/Theme
    2. Key Points
    3. Important Details
    4. Conclusions"""

    try:
        completion = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=2048
        )
        return completion.choices[0].message.content
    except Exception as e:
        print(f"Error in summarization: {e}")
        return None

## Main Function to Summarize YouTube Video

In [ ]:
def summarize_youtube_video(url):
    """Main function to summarize a YouTube video"""
    # Extract video ID
    video_id = extract_video_id(url)
    if not video_id:
        return "Invalid YouTube URL"
    
    # Get transcript
    transcript = get_transcript(video_id)
    if not transcript:
        return "Could not retrieve transcript"
    
    # Generate summary
    summary = summarize_text(transcript)
    if not summary:
        return "Could not generate summary"
    
    return summary

## Example Usage

To use this summarizer, you'll need to:
1. Create a `.env` file in the same directory as this notebook
2. Add your Groq API key to the `.env` file: `GROQ_API_KEY=your_api_key_here`

Then you can use the summarizer as shown below:

In [ ]:
# Example usage
youtube_url = "https://www.youtube.com/watch?v=your_video_id"
summary = summarize_youtube_video(youtube_url)
print(summary)